In [186]:
import gensim as gs
from nltk.corpus import wordnet as wn
import pandas as pd
import os
import sys
from progressbar import ProgressBar
import nltk
import re
from nltk import pos_tag as ptag
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [144]:
tm_df = pd.read_csv('index.csv',delimiter=';',index_col = False)
#print(tm_df.head(5))
print(tm_df.loc[tm_df['filename'] == "1079196.txt"])

      filename                                             prompt grade
2  1079196.txt  Do you agree or disagree with the following st...  high


In [145]:
# record the file names and topics in a dict
dtf = {}
for i, row in tm_df.iterrows():
    prompt = row['prompt'].replace('\t', '').replace('\t','').replace('.', '. ').replace('?', '. ').strip() 
    if row['filename'] not in dtf:
        dtf[row['filename']] = None
    dtf[row['filename']] = prompt
#print(dtf)

In [146]:
# essay files location
data_path_train = "../input/training/"
filename="1079196.txt"
f_ = open(data_path_train+filename)
text=f_.read().replace('\t', '').replace('\t','').replace('.', '. ').strip()
#print(text)



In [147]:
prompt_string = dtf[filename]
#print(prompt_string)

In [148]:
# get all nouns and action words from the topic
action_words = []
def tag(prompt_string):
    prompt_string = re.sub('[\,.!?()``:-;'']', '', prompt_string)
    parsed = ptag(nltk.word_tokenize(prompt_string))
    #print(parsed)
    
    for p in parsed:
        #if p[1] in ('VB','NNS','VBP','VBG','VBZ','VBD','VBN','NN','NNP','RB'):
        if p[1] not in ('TO','IN','CC','PRP$','PRP','DT'):
            action_words.append(p[0].lower())
    
    return action_words
action_words = tag(prompt_string) 
print(action_words)


['do', 'agree', 'disagree', 'following', 'statement', 'most', 'advertisements', 'make', 'products', 'seem', 'much', 'better', 'really', 'are', 'use', 'specific', 'reasons', 'examples', 'support', 'answer']


In [149]:
# for each action word get all related words of each action word and check them against all the words in the essay
# example
sysnet_words = []
for action in action_words:
    action = wn.synsets(action)
    for synset in action:
        for lemma in synset.lemmas():
            sysnet_words.append(lemma.name())
        
#print(sysnet_words)


In [150]:
# take set intersection of words with the essay and action word sysnets
essay_words  = tag(text)
#print(essay_words)

intersect = set(essay_words).intersection(set(sysnet_words))
print(intersect,len(intersect))


{'exist', 'much', 'really', 'seem', 'often', 'well', 'have', 'get', 'product', 'disagree', 'do', 'make', 'clear', 'most', 'following', 'statement', 'live', 'be', 'better', 'truly', 'best', 'use', 'honest', 'answer', 'agree', 'specific', 'effective', 'advertising', 'advertisement', 'are', 'support'} 31


In [160]:
# textblob_sentiment with respect to essay
essay_blob = TextBlob(text)
print(essay_blob.sentiment.subjectivity)
# give subjectivity more preference as it reflects on the opinion of the author


0.5520125786163521


In [152]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in [text]] 

In [161]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(doc_clean)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [154]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

In [155]:
top = ldamodel.print_topics(num_topics=1, num_words=5)[0][1]

In [156]:
print(res[0][1])

0.007*"product" + 0.007*"company" + 0.007*"advertisement" + 0.007*"advertising" + 0.007*"rely"


In [157]:
top = res[0][1]
ls = re.findall(r'"(.*?)"',top)
print(set(re.findall(r'"(.*?)"',top)).intersection(set(sysnet_words).union(set(action_words))))

{'product', 'rely', 'advertisement', 'advertising', 'company'}


In [196]:
train = pd.read_csv("../dump/results.csv",names=["Filename","A","B","C1","C2","C3","D1","D2a","D2b","D2c","Final_Score","Grade","Cohort"],index_col=False)
train.replace({"high":1,"low":0},inplace=True)
train_values = pd.read_csv("../dump/trainvalues.csv")
train['A'] = train_values['length_train']
train['C1'] = train_values['c1_train']
mms = MinMaxScaler()


In [197]:
train.head(5)

,Filename,A,B,C1,C2,C3,D1,D2a,D2b,D2c,Final_Score,Grade,Cohort
0,1728162.txt,14.0,16,0,9,0,0,18,0.565667,2,0,1,0
1,1804641.txt,10.0,22,1,6,0,0,4,0.579167,1,0,0,0
2,1469088.txt,17.0,18,1,7,0,0,13,0.331481,0,0,0,0
3,755753.txt,11.0,11,1,11,0,0,14,0.421239,3,0,1,0
4,1346734.txt,6.0,11,0,5,0,0,10,0.510799,1,0,0,0


In [205]:
train_trans = train.copy()
x_train = pd.DataFrame(columns=['A','B','C1','C2',"D2a","D2b","D2c"])
x_train[['A','B','C1','C2',"D2a","D2b","D2c"]] = mms.fit_transform(train_trans[['A','B','C1','C2',"D2a","D2b","D2c"]])
#x_train['D2'] = 0
x_train['D2_temp'] = x_train['D2a'].astype(float) + x_train['D2b'].astype(float) + x_train['D2c'].astype(float)
x_train['D2'] = x_train['D2_temp'].apply(lambda x:x/3)
x_train.drop(['D2a','D2b','D2c','D2_temp'], axis = 1, inplace=True)


ValueError: shape mismatch: value array of shape (69,7) could not be broadcast to indexing result of shape (7,0)

In [195]:
x_train = mms.fit_transform(train[['A','B','C1','C2',"D2a","D2b","D2c"]])
temp = (x_train[:,-1] + x_train[:,-2] + x_train[:,-3])/3.0
print(x_train)
x_train = np.delete(x_train, np.s_[3:6], 1)
print(x_train)
x_train = np.c_[x_train,temp]
print(x_train)


[[0.47826087 0.05       0.         0.23529412 0.77272727 0.18754754
  0.4       ]
 [0.30434783 0.325      0.5        0.23529412 0.27272727 0.3680284
  0.4       ]
 [0.60869565 0.375      0.5        0.47058824 0.40909091 0.1110236
  0.8       ]
 [0.34782609 0.2        0.5        0.23529412 0.13636364 0.11288955
  0.        ]
 [0.13043478 0.125      0.         0.52941176 0.59090909 0.41056196
  0.6       ]
 [0.56521739 0.475      0.         0.11764706 0.22727273 0.4266224
  0.4       ]
 [0.43478261 0.15       0.         0.29411765 0.45454545 1.
  0.6       ]
 [0.13043478 0.175      0.         0.52941176 0.31818182 0.48260701
  0.4       ]
 [0.39130435 0.5        0.         0.29411765 0.         0.49913973
  0.4       ]
 [0.60869565 0.175      0.         0.         0.22727273 0.16729026
  0.2       ]
 [0.34782609 0.125      0.         0.29411765 0.68181818 0.25102921
  0.8       ]
 [0.52173913 0.2        0.         0.35294118 0.59090909 0.11624824
  0.6       ]
 [0.06521739 0.75       0. 